## Calculating Stock Beta, Volatility, and More
#### http://gouthamanbalaraman.com/blog/calculating-stock-beta.html


In [10]:
from pandas_datareader import data as wb
import datetime as dt
import numpy as np
import pandas as pd

In [11]:
# Grab time series data for 5-year history for the stock (here AAPL)
# and for S&P-500 Index

sdate = dt.datetime(2008, 12, 30)
edate = dt.datetime(2013, 12, 30)


acao1 = 'BRFS3.SA'
acao2 = 'WEGE3.SA'
acao3 = 'LAME4.SA'
indx = '^BVSP'

df = wb.DataReader(acao1, data_source = 'yahoo',start = sdate,end = edate)
dfb = wb.DataReader(indx, data_source = 'yahoo',start = sdate,end = edate)

In [12]:
# create a time-series of monthly data points
rts = df.resample('D').last()
rbts = dfb.resample('D').last()
dfsm = pd.DataFrame({'s_adjclose' : rts['Adj Close'],
                        'b_adjclose' : rbts['Adj Close']},
                        index=rts.index)
rts.head(3)

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2008-12-30,15.45,14.87,15.275,14.87,1401600.0,13.783742
2008-12-31,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-01,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# compute returns
dfsm[['s_returns','b_returns']] = dfsm[['s_adjclose','b_adjclose']]/\
    dfsm[['s_adjclose','b_adjclose']].shift(1) -1
dfsm = dfsm.dropna()
covmat = np.cov(dfsm["s_returns"],dfsm["b_returns"])
#dfsm.head()
covmat

array([[0.00045847, 0.00014177],
       [0.00014177, 0.00027877]])

In [5]:
# calculate measures now
beta = covmat[0,1]/covmat[1,1]
alpha = np.mean(dfsm["s_returns"])-beta*np.mean(dfsm["b_returns"])

In [6]:
# r_squared     = 1. - SS_res/SS_tot
ypred = alpha + beta * dfsm["b_returns"]
SS_res = np.sum(np.power(ypred-dfsm["s_returns"],2))
SS_tot = covmat[0,0]*(len(dfsm)-1) # SS_tot is sample_variance*(n-1)
r_squared = 1. - SS_res/SS_tot

In [7]:
# 5- year volatiity and 1-year momentum
volatility = np.sqrt(covmat[0,0])
momentum = np.prod(1+dfsm["s_returns"].tail(250).values) -1

In [8]:
# annualize the numbers
prd = 250. # used monthly returns; 12 periods to annualize
alpha = alpha*prd
volatility = volatility*np.sqrt(prd)

print (beta,alpha, r_squared, volatility, momentum)

0.5407447557168445 0.33371051083091546 0.1713418355110311 0.30410089713256017 0.4116868013603643
